In [4]:
import numpy as np
import unittest
from math import cos, sin

import numpy as np
from xdyn.core import BodyBuilder, BodyPtr, BodyStates, EnvironmentAndFrames
from xdyn.core.io import YamlRotation
from xdyn.data.mesh import two_triangles
from xdyn.force import ConstantForceModel
from xdyn.ssc.kinematics import EulerAngles as SscEulerAngles
from xdyn.ssc.random import DataGenerator


def get_input() -> str:
    """Create a YAML data model string"""
    return """
    model: constant force
    frame: NED
    x: {value: 0.5, unit: m}
    y: {value: -0.2, unit: m}
    z: {value: -440, unit: m}
    X: {value: 10, unit: kN}
    Y: {value: 20, unit: kN}
    Z: {value: 30, unit: kN}
    K: {value: 100, unit: kN*m}
    M: {value: 200, unit: kN*m}
    N: {value: 300, unit: kN*m}
    """


def get_env() -> EnvironmentAndFrames:
    env = EnvironmentAndFrames()
    env.rot = YamlRotation("angle", ["z", "y'", "x''"])
    env.rho = 1024
    return env


def get_body(name: str) -> BodyPtr:
    rot = YamlRotation("angle", ["z", "y'", "x''"])
    builder = BodyBuilder(rot)
    return builder.build_for_test(name, two_triangles(), 0, 0.0, rot, 0.0, False)


def get_states(
    env: EnvironmentAndFrames, body_name: str, phi: float, theta: float, psi: float
) -> BodyStates:
    """Create a body state variable with one record"""
    body = get_body(body_name)
    states = body.get_states()
    states.convention = YamlRotation("angle", ["z", "y'", "x''"])
    quaternion = states.convert_to_quaternion(SscEulerAngles(phi, theta, psi), states.convention)
    states.x.record(0, 0.1)
    states.y.record(0, 2.04)
    states.z.record(0, 6.28)
    states.u.record(0, 0.45)
    states.v.record(0, 0.01)
    states.w.record(0, 5.869)
    states.p.record(0, 0.23)
    states.q.record(0, 0)
    states.r.record(0, 0.38)
    states.qr.record(0, quaternion[0])  # To improve
    states.qi.record(0, quaternion[1])
    states.qj.record(0, quaternion[2])
    states.qk.record(0, quaternion[3])
    body.update_kinematics(states.get_current_state_values(0), env.k)
    return states


ctm_x = lambda alpha: np.array(
    [[1.0, 0.0, 0.0], [0.0, cos(alpha), sin(alpha)], [0.0, -sin(alpha), cos(alpha)]]
)
ctm_y = lambda alpha: np.array(
    [[cos(alpha), 0.0, -sin(alpha)], [0.0, 1.0, 0.0], [sin(alpha), 0.0, cos(alpha)]]
)
ctm_z = lambda alpha: np.array(
    [[cos(alpha), sin(alpha), 0.0], [-sin(alpha), cos(alpha), 0.0], [0.0, 0.0, 1.0]]
)


In [5]:
data = ConstantForceModel.parse(get_input())

In [7]:
print(data.x)

0.5
